In [ ]:
import pandas as pd #
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./data/Landslide3.csv"
device = "102"
save_path = f"./data/dev{device}_prepared.csv"
#save_path = f"data/devAll2_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

def clean_and_format_timestamp(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('(ICT)', '').strip()
    try:
        dt = pd.to_datetime(val)
        return dt.isoformat()
    except Exception:
        return None  

df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])

df = df.drop(columns=['unixt', 'gateway time', 'old time', 'risk_level', 'confidence'], errors='ignore')
df = df[df['devID'] != 218499049.0]
df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
df = df[df['lng'] > 15.0]
df = df[df['soil'] != 0.0]
df = df[df['rain'] != -1.0]

df = df[df['devID'] == int(device)]
df

C:\Users\ahmad\AppData\Local\Temp\ipykernel_11964\3411799755.py:13: DtypeWarning: Columns (9,10,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
1,2025-05-02 11:42:00.384119,102.0,22.200001,0.0,34.549099,61.175941,3.323395,8.637964,99.898933
3,2025-05-02 11:42:11.034615,102.0,22.200001,0.0,34.506374,61.542152,14.766250,8.637963,99.898933
5,2025-05-02 11:42:21.675224,102.0,22.200001,0.0,34.485012,61.767223,3.242806,8.637963,99.898933
7,2025-05-02 11:42:32.443581,102.0,22.200001,0.0,34.477001,62.366138,6.375742,8.637964,99.898933
9,2025-05-02 11:42:43.142656,102.0,22.200001,0.0,34.447620,60.361488,19.010500,8.637964,99.898933
...,...,...,...,...,...,...,...,...,...
383607,2025-07-22 19:39:15.911000,102.0,23.299999,0.0,28.658354,73.837112,2.621462,8.637943,99.898926
383608,2025-07-22 19:39:22.155000,102.0,23.299999,0.0,28.669035,74.106049,2.609490,8.637943,99.898926
383610,2025-07-22 19:39:35.152000,102.0,23.299999,0.0,28.653008,73.873352,5.533101,8.637944,99.898926
383612,2025-07-22 19:39:48.552000,102.0,23.299999,0.0,28.631645,73.905777,6.762322,8.637944,99.898926


In [2]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
count,188893,188893.0,188893.000000,188893.000000,188893.000000,188893.000000,188893.000000,188893.000000,188893.000000
mean,2025-06-10 16:27:04.315065600,102.0,44.319324,0.153583,30.010319,74.138998,5.321778,8.637945,99.898928
min,2025-05-02 11:42:00.384119,102.0,20.200001,0.000000,24.238958,43.418247,0.000000,8.637330,99.883331
25%,2025-05-30 04:07:41.435315968,102.0,27.400000,0.000000,27.368584,67.350113,2.684892,8.637937,99.898918
50%,2025-06-13 08:09:29.560000,102.0,48.400002,0.000000,28.615623,78.470131,3.893713,8.637944,99.898926
75%,2025-06-27 12:36:45.910000128,102.0,57.700001,0.000000,33.101780,81.088959,4.346031,8.637954,99.898941
max,2025-07-22 19:39:55.017000,102.0,64.500000,34.366199,39.331657,88.510567,32.062031,8.638361,99.900078
std,NaN,0.0,14.902299,1.638992,3.297162,9.293528,4.981407,0.000020,0.000045


In [3]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
1,2025-05-02 11:42:00.384119,102.0,22.200001,0.0,34.549099,61.175941,3.323395,8.637964,99.898933
3,2025-05-02 11:42:11.034615,102.0,22.200001,0.0,34.506374,61.542152,14.766250,8.637963,99.898933
5,2025-05-02 11:42:21.675224,102.0,22.200001,0.0,34.485012,61.767223,3.242806,8.637963,99.898933
7,2025-05-02 11:42:32.443581,102.0,22.200001,0.0,34.477001,62.366138,6.375742,8.637964,99.898933
9,2025-05-02 11:42:43.142656,102.0,22.200001,0.0,34.447620,60.361488,19.010500,8.637964,99.898933
...,...,...,...,...,...,...,...,...,...
383607,2025-07-22 19:39:15.911000,102.0,23.299999,0.0,28.658354,73.837112,2.621462,8.637943,99.898926
383608,2025-07-22 19:39:22.155000,102.0,23.299999,0.0,28.669035,74.106049,2.609490,8.637943,99.898926
383610,2025-07-22 19:39:35.152000,102.0,23.299999,0.0,28.653008,73.873352,5.533101,8.637944,99.898926
383612,2025-07-22 19:39:48.552000,102.0,23.299999,0.0,28.631645,73.905777,6.762322,8.637944,99.898926


In [4]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.set_index('timestamp')
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek
df = df.reset_index() 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188893 entries, 0 to 188892
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   timestamp    188893 non-null  datetime64[ns]
 1   devID        188893 non-null  float64       
 2   soil         188893 non-null  float64       
 3   rain         188893 non-null  float64       
 4   temp         188893 non-null  float64       
 5   humi         188893 non-null  float64       
 6   geo          188893 non-null  float64       
 7   lat          188893 non-null  float64       
 8   lng          188893 non-null  float64       
 9   hour         188893 non-null  int32         
 10  day_of_week  188893 non-null  int32         
dtypes: datetime64[ns](1), float64(8), int32(2)
memory usage: 14.4 MB


In [6]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [7]:
df = df.dropna(subset=['lng']).reset_index(drop=True)

In [8]:
nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [9]:
df = df.dropna()

nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("df102")
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

df102
📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [10]:

df.to_csv(f"{save_path}", index=False)

print("✅ บันทึกเรียบร้อยแล้ว")

✅ บันทึกเรียบร้อยแล้ว
